In [ ]:

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import requests

app = dash.Dash(__name__)

url = 'http://127.0.0.1:5000/'  # Replace with your API endpoint
headers = {}  # Add any headers needed for your API
payload = {}  # Add any payload needed for your API

app.layout = html.Div([
    html.H1("Trip Information"),

    html.Button('Get Trips', id='get-trips-button', n_clicks=0),

    html.Div(id='trips-output-container', children=[]),

    html.Hr(),  # Add a horizontal line

    html.H1("Expense Statistics"),

    html.Label("Average Accommodation Expense:"),
    html.Div(id='average-expense-output'),

    html.Button('Get Average Expense', id='get-average-expense-button', n_clicks=0),

    html.Hr(),  # Add a horizontal line

    html.H1("Expense Search"),

    html.H1("Expense Search App"),
    html.Label("Enter the minimum amount:"),
    dcc.Input(id='min-amount', type='number', value=0),
    html.Label("Enter the maximum amount:"),
    dcc.Input(id='max-amount', type='number', value=1000),
    html.Button('Search Expenses', id='search-button', n_clicks=0),
    html.Div(id='output-container')
])

@app.callback(
    Output('trips-output-container', 'children'),
    [Input('get-trips-button', 'n_clicks')]
)
def get_trips(n_clicks):
    if n_clicks == 0:
        return []

    url = 'http://127.0.0.1:5000/'  # Replace with your actual API URL
    headers = {"Authorization": "Bearer YourAccessToken"}  # Replace with your actual access token

    response = requests.get(url + 'trips', headers=headers)

    if response.status_code == 200:
        trips = response.json()
        trips_list = [html.P(f"Trip ID: {trip['id']}, Traveler: {trip['traveler_name']}, Destination: {trip['destination']}") for trip in trips]
        return trips_list
    else:
        return [html.P(f"Error: {response.status_code} - {response.text}")]


   
@app.callback(
    Output('average-expense-output', 'children'),
    [Input('get-average-expense-button', 'n_clicks')]
)
def get_average_expense(n_clicks):
    if n_clicks == 0:
        return []

    url = "http://127.0.0.1:5000/expenses/accommodation/average"
    
    response = requests.get(url)

    if response.status_code == 200:
        average_expense = response.json()
        return [html.P(f"Average Accommodation Expense: {average_expense}")]
    else:
        return [html.P(f"Error: {response.status_code} - {response.text}")]


if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


@app.callback(
    Output('output-container', 'children'),
    [Input('search-button', 'n_clicks')],
    [dash.dependencies.State('min-amount', 'value'),
     dash.dependencies.State('max-amount', 'value')]
)
def update_output(n_clicks, min_amount, max_amount):
    if n_clicks > 0:
        # Perform the API request to get expenses in the specified range
        url = "http://localhost:5000/expenses/search"
        params = {"amount_min": min_amount, "amount_max": max_amount}
        response = requests.get(url, params=params)

        if response.status_code == 200:
            expenses = response.json()

            # Display the expenses in an HTML table
            expenses_table = [
                html.H3("Expenses in the specified range:"),
                html.Table(
                    # Header
                    [html.Tr([html.Th(key) for key in expenses[0]])] +
                    # Body
                    [html.Tr([html.Td(expense[key]) for key in expense]) for expense in expenses]
                )
            ]

            return expenses_table
        else:
            return f"Error: {response.status_code} - {response.text}"

if __name__ == '__main__':


    app.run_server(debug=True, port=8051)